# Face Recognition - Generate Eigenface Classifiers

This notebook tests face detection and recognition from sample images

In [1]:
import cv2, cv2.face
import os, os.path
import numpy as np
from PIL import Image

In [2]:
# constants
CASCADE_PATH = './data/'
CASCADE_NAME = 'haarcascade_frontalface_default.xml'
SAMPLE_TRAINING = './data/sample_training/'
SAMPLE_TESTING = './data/sample_testing/'

In [3]:
#vSAMPLE_TRAINING = '/home/rico/datasets/lfw/raw/'

In [4]:
# Viola-Jones classifier for Haar feature extraction
cascader_file = os.path.join(CASCADE_PATH, CASCADE_NAME)
cascader = cv2.CascadeClassifier(cascader_file)
assert(not cascader.empty())

cascader_kwargs = { # setup detection args here
    'scaleFactor': 1.1,
    'minNeighbors': 2,
    'flags': 2
}

In [5]:
from util import get_images_and_labels

In [6]:
# setup label dictionary and testing set

label_dict = {}
label_array = []
    
testing_set = get_images_and_labels(SAMPLE_TESTING, cascader=cascader, detect_faces=False, debug=False, debug_faces=True, debug_accuracy=True)


Accuracy: 100.00000%


In [7]:
def perform_training(recognizer, **kwargs):
    training_set = get_images_and_labels(SAMPLE_TRAINING, **kwargs)
    images, labels = training_set
    
    label_dict.clear()
    label_array.clear()
    
    for label in labels:
        if label not in label_dict:
            label_dict[label] = len(label_dict)
            label_array.append(label)
    print(label_dict)
    print([label_dict[label] for label in labels])
    recognizer.train(images, np.array([label_dict[label] for label in labels]))
    print('Trained %d images\n' % len(images))

In [8]:
def perform_testing(recognizer):
    %%time
    # Perform testing

    images, labels = testing_set
    zipped = zip(images, labels)
    print('Performing %d tests\n' % len(images))

    correct = 0
    total = 0

    for image, expected in zipped:
        if expected not in label_dict:
            print('No face found for %s' % expected)
        else:
            eid = label_dict[expected]
            lid, confidence = recognizer.predict(image)
            if lid != -1:
                actual = label_array[lid]
                if eid == lid:
                    correct += 1
                    print('%s is correctly recognized with confidence %.10f' % (expected, confidence))
                else:
                    print('%s incorrect (recognized as %s with confidence %.10f)' % (expected, actual, confidence))
            else:
                print('No face found for %s' % expected)
        total += 1

    print()
    print('Accuracy: %.2f%%' % (100 * (correct / max(1, total))))
    print('Correct:  %d' % correct)
    print('Wrong:    %d' % (total - correct))

# 1. EigenFaces

In [9]:
# face recognizer
recognizer = cv2.face.createEigenFaceRecognizer()

In [10]:
%%time
# Perform the training
perform_training(recognizer, detect_faces=False, cascader=cascader)

Accuracy: 100.00000%
{'George_Clooney': 7, 'Hosni_Mubarak': 0, 'Fernando_Gonzalez': 6, 'Colin_Farrell': 8, 'Charles_Taylor': 9, 'Hugh_Grant': 5, 'Jeong_Se-hyun': 1, 'Jesse_Jackson': 4, 'Heizo_Takenaka': 3, 'Aaron_Peirsol': 2}
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 580 ms, sys: 24 ms, total: 604 ms
Wall time: 600 ms


In [11]:
perform_testing(recognizer)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs
Performing 10 tests

Hosni_Mubarak incorrect (recognized as Hugh_Grant with confidence 7709.3543340320)
Jeong_Se-hyun incorrect (recognized as Jesse_Jackson with confidence 9224.6342818637)
Aaron_Peirsol incorrect (recognized as Jeong_Se-hyun with confidence 7707.7644787612)
Heizo_Takenaka is correctly recognized with confidence 7660.4060534007
Jesse_Jackson incorrect (recognized as George_Clooney with confidence 7555.9205502438)
Hugh_Grant is correctly recognized with confidence 7110.0422084874
Fernando_Gonzalez is correctly recognized with confidence 7212.4169510010
George_Clooney is correctly recognized with confidence 8450.9567331031
Colin_Farrell incorrect (recognized as Hugh_Grant with confidence 7860.5953948817)
Charles_Taylor incorrect (recognized as Fernando_Gonzalez with confidence 9513.5099176005)

Accuracy: 40.00%
Correct:  4
Wrong:    6


# 2. FisherFaces

In [12]:
# face recognizer
recognizer = cv2.face.createFisherFaceRecognizer()

In [13]:
%%time
perform_training(recognizer, debug=True, debug_faces=False, detect_faces=False)

Hosni_Mubarak
	Hosni_Mubarak_0007.png [1 face]
	Hosni_Mubarak_0003.png [1 face]
	Hosni_Mubarak_0004.png [1 face]
	Hosni_Mubarak_0008.png [1 face]
	Hosni_Mubarak_0005.png [1 face]
	Hosni_Mubarak_0001.png [1 face]
	Hosni_Mubarak_0006.png [1 face]
	Hosni_Mubarak_0002.png [1 face]
[Added 8 images]
Jeong_Se-hyun
	Jeong_Se-hyun_0007.png [1 face]
	Jeong_Se-hyun_0001.png [1 face]
	Jeong_Se-hyun_0008.png [1 face]
	Jeong_Se-hyun_0003.png [1 face]
	Jeong_Se-hyun_0004.png [1 face]
	Jeong_Se-hyun_0002.png [1 face]
	Jeong_Se-hyun_0005.png [1 face]
	Jeong_Se-hyun_0006.png [1 face]
[Added 16 images]
Aaron_Peirsol
	Aaron_Peirsol_0003.png [1 face]
	Aaron_Peirsol_0001.png [1 face]
	Aaron_Peirsol_0002.png [1 face]
[Added 19 images]
Heizo_Takenaka
	Heizo_Takenaka_0002.png [1 face]
	Heizo_Takenaka_0007.png [1 face]
	Heizo_Takenaka_0001.png [1 face]
	Heizo_Takenaka_0006.png [1 face]
	Heizo_Takenaka_0004.png [1 face]
	Heizo_Takenaka_0008.png [1 face]
	Heizo_Takenaka_0003.png [1 face]
	Heizo_Takenaka_0005.png 

In [14]:
%%time
perform_testing(recognizer)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.34 µs
Performing 10 tests

Hosni_Mubarak incorrect (recognized as Hugh_Grant with confidence 1358.7258628299)
Jeong_Se-hyun incorrect (recognized as Hosni_Mubarak with confidence 1960.9871499361)
Aaron_Peirsol incorrect (recognized as Hosni_Mubarak with confidence 1491.9996239568)
Heizo_Takenaka incorrect (recognized as Colin_Farrell with confidence 2196.2782370959)
Jesse_Jackson incorrect (recognized as Hugh_Grant with confidence 1464.6791591270)
Hugh_Grant is correctly recognized with confidence 1687.1694825283
Fernando_Gonzalez is correctly recognized with confidence 1754.4300679463
George_Clooney is correctly recognized with confidence 1596.7317624799
Colin_Farrell incorrect (recognized as Fernando_Gonzalez with confidence 1637.2118716964)
Charles_Taylor incorrect (recognized as Hosni_Mubarak with confidence 1644.8235677224)

Accuracy: 30.00%
Correct:  3
Wrong:    7
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 4.9

# 2. Local Binary Patterns

In [15]:
# face recognizer
recognizer = cv2.face.createLBPHFaceRecognizer()

In [16]:
%%time
perform_training(recognizer, cascader=cascader)

Accuracy: 94.66667%
{'George_Clooney': 7, 'Hosni_Mubarak': 0, 'Fernando_Gonzalez': 6, 'Colin_Farrell': 8, 'Charles_Taylor': 9, 'Hugh_Grant': 5, 'Jeong_Se-hyun': 1, 'Jesse_Jackson': 4, 'Heizo_Takenaka': 3, 'Aaron_Peirsol': 2}
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 71 images

CPU times: user 2.62 s, sys: 24 ms, total: 2.64 s
Wall time: 1.15 s


In [17]:
%%time
perform_testing(recognizer)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.34 µs
Performing 10 tests

Hosni_Mubarak incorrect (recognized as Charles_Taylor with confidence 97.1662617714)
Jeong_Se-hyun incorrect (recognized as Charles_Taylor with confidence 85.1684303021)
Aaron_Peirsol incorrect (recognized as Charles_Taylor with confidence 79.9038326453)
Heizo_Takenaka incorrect (recognized as Charles_Taylor with confidence 85.1785419217)
Jesse_Jackson incorrect (recognized as Charles_Taylor with confidence 66.3214017200)
Hugh_Grant incorrect (recognized as Charles_Taylor with confidence 81.8931253215)
Fernando_Gonzalez is correctly recognized with confidence 71.3826883620
George_Clooney incorrect (recognized as Charles_Taylor with confidence 81.4138628045)
Colin_Farrell incorrect (recognized as Charles_Taylor with confidence 78.3390875249)
Charles_Taylor is correctly recognized with confidence 80.2390735344

Accuracy: 20.00%
Correct:  2
Wrong:    8
CPU times: user 76 ms, sys: 8 ms, total: 84 ms
Wall t